## Import all the necessary libraries 

In [1]:
import numpy as np
import pandas as pd

In [2]:
import cv2
import pytesseract
import numpy as np
from fuzzywuzzy import fuzz
from sklearn.metrics.pairwise import cosine_similarity
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import models
from torchvision.models import resnet18, ResNet18_Weights
from PIL import Image

In [3]:
class SignatureEmbedder(nn.Module):
    def __init__(self):
        super(SignatureEmbedder, self).__init__()

        # Load pretrained ResNet with proper weights
        weights = ResNet18_Weights.DEFAULT
        self.model = resnet18(weights=weights)
        self.model.fc = nn.Identity()  # Remove final classification layer

    def forward(self, x):
        return self.model(x)
    
def preprocess_signature(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    # Step 2: Resize to 224x224 (what ResNet expects)
    img = cv2.resize(img, (224, 224))
    # Step 3: Convert 1-channel grayscale to 3-channel RGB
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    # Step 4: Convert to PIL Image
    img_pil = Image.fromarray(img)
    transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize(mean=[0.5, 0.5, 0.5], 
                                                     std=[0.5, 0.5, 0.5])])
    img_tensor = transform(img_pil).unsqueeze(0)
    return img_tensor

# Signature Comparison
def compare_signatures(sig_path1, sig_path2, model):
    img1 = preprocess_signature(sig_path1)
    img2 = preprocess_signature(sig_path2)

    with torch.no_grad():
        emb1 = model(img1)
        emb2 = model(img2)
    similarity = cosine_similarity(emb1.numpy(), emb2.numpy())[0][0]
    return similarity

# Main Verification function
def verify_customer(reference_signature_path, test_signature_path):
    model = SignatureEmbedder()
    sign_score = compare_signatures(reference_signature_path, test_signature_path, model)
    print(f"Signature Similarity Score: {sign_score}")

    if sign_score > 0.85:
        print("✅ Signature Verified")
    else:
        print("❌ Verification failed")


In [4]:

# Example usage
verify_customer(
    reference_signature_path="D:/signature_kyc/signature/sign_sample/signature (5).png",
    test_signature_path="D:/signature_kyc/signature/sign_sample/signature (6).png"
)

Signature Similarity Score: 0.998323917388916
✅ Signature Verified


In [8]:
import os
print(os.path.exists("D:/signature_kyc/signaturesign_sample/signature (5).png"))  # should return True


False
